In [1]:
import pandas as pd
from comet import download_model, load_from_checkpoint

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-25 12:28:19.100131: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 12:28:19.162759: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 12:28:21.444748: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different n

In [2]:
comet_da_model_path = download_model("wmt21-comet-da")
comet_da_model = load_from_checkpoint(comet_da_model_path)

comet_mqm_model_path = download_model("wmt21-comet-mqm")
comet_mqm_model = load_from_checkpoint(comet_mqm_model_path)

wmt21-comet-da is already in cache.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../root/.cache/torch/unbabel_comet/wmt21-comet-da/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
wmt21-comet-mqm is already in cache.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../root/.cache/torch/unbabel_comet/wmt21-comet-mqm/checkpoints/model.ckpt`
Encoder model frozen.


In [3]:
from evaluate import load
cometm = load("comet")

PyTorch version 2.5.1+cu121 available.
TensorFlow version 2.20.0 available.
Fetching 5 files: 100%|████████████████████| 5/5 [00:00<00:00, 60436.66it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.


In [4]:
da = {}
mqm = {}
comet = {}

## Original

In [5]:
file_name = "shalaka_original_outputs.csv"
mode = "original"

In [6]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
src_sentences = df['src']
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.11it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [7]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [8]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

references = [[r1, r2, r3] for r1, r2, r3 in zip(df['gt'], df['gemini'], df['cfilt'])]

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'original': -0.0009308650710240558
Average COMET-MQM for 'original': 0.030412373512431427
Average COMET for 'original': 0.7815145642301182


## Only Punctuations

In [9]:
file_name = "shalaka_only_punct_outputs.csv"
mode = "only_punct"

In [10]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.12it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [11]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokeniz

In [12]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'only_punct': -0.017625263305725874
Average COMET-MQM for 'only_punct': 0.029490288243525557
Average COMET for 'only_punct': 0.7733062818462466


## Without Punctuations

In [13]:
file_name = "shalaka_without_punct_outputs.csv"
mode = "without_punct"

In [14]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.01it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [15]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [16]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'without_punct': -0.025836327665106015
Average COMET-MQM for 'without_punct': 0.028871826758539235
Average COMET for 'without_punct': 0.7708970168490469


## Combined

In [17]:
file_name = "shalaka_combined_outputs.csv"
mode = "combined"

In [18]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:07<00:00,  1.03s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [19]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [20]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'combined': -0.011757608201285755
Average COMET-MQM for 'combined': 0.029478729478325003
Average COMET for 'combined': 0.7749270211767268


## Combined - LR & Epochs Changed

In [21]:
file_name = "shalaka_combined_le_outputs.csv"
mode = "combined_le"

In [22]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.08it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [23]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [24]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'combined_le': -0.007987807642806459
Average COMET-MQM for 'combined_le': 0.029587087618118083
Average COMET for 'combined_le': 0.7836923915662884


## Combined - LR & Epochs & Dataset Changed

In [25]:
file_name = "shalaka_combined_led_outputs.csv"
mode = "combined_led"

In [26]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:11<00:00,  1.68s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [27]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [28]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'combined_led': -0.010800619075005805
Average COMET-MQM for 'combined_led': 0.029474133401419276
Average COMET for 'combined_led': 0.7844210015034969


## GPT-5

In [29]:
file_name = "shalaka_gpt_outputs.csv"
mode = "gpt"

In [30]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.03it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [31]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [32]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'gpt': -0.049362684393094644
Average COMET-MQM for 'gpt': 0.02795883163344115
Average COMET for 'gpt': 0.7372043658921749


## BERT

In [33]:
file_name = "approach1_bert_outputs.csv"
mode = "bert"

In [34]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.15it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [35]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [36]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'bert': 0.011362060757905797
Average COMET-MQM for 'bert': 0.031046148872485867
Average COMET for 'bert': 0.8049476095187811


## MPNet

In [37]:
file_name = "approach1_mpnet_outputs.csv"
mode = "mpnet"

In [38]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:06<00:00,  1.13it/s]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [39]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [40]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 'mpnet': 0.013834364893328812
Average COMET-MQM for 'mpnet': 0.031341622531827956
Average COMET for 'mpnet': 0.8059657333809652


## T5

In [41]:
file_name = "approach1_t5_outputs.csv"
mode = "t5"

In [42]:
df = pd.read_csv(file_name)

predictions = df['prediction'].tolist()
references = df[['gt', 'gemini', 'cfilt']].values.tolist()

data_for_comet = [
    {
        "src": src,
        "mt": pred,
        "ref": refs
    }
    for src, pred, refs in zip(df['src'], predictions, references)
]

da_scores = comet_da_model.predict(data_for_comet, batch_size=8, gpus=0)
mqm_scores = comet_mqm_model.predict(data_for_comet, batch_size=8, gpus=0)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|████████████████| 7/7 [00:07<00:00,  1.05s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:166: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 10

In [43]:
scores_gt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gt'].tolist()
)['mean_score']

scores_gemini = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['gemini'].tolist()
)['mean_score']

scores_cfilt = cometm.compute(
    sources=df['src'].tolist(),
    predictions=df['prediction'].tolist(),
    references=df['cfilt'].tolist()
)['mean_score']

comet_score = (scores_gt + scores_gemini + scores_cfilt) / 3

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
💡 Tip: For seamless

In [44]:
da_scores = pd.DataFrame(da_scores)['scores']
mqm_scores = pd.DataFrame(mqm_scores)['scores']

overall_da = da_scores.mean()
overall_mqm = mqm_scores.mean()

da[mode] = overall_da
mqm[mode] = overall_mqm
comet[mode] = comet_score

print(f"Average COMET-DA for '{mode}':", overall_da)
print(f"Average COMET-MQM for '{mode}':", overall_mqm)
print(f"Average COMET for '{mode}':", comet_score)

Average COMET-DA for 't5': 0.01578567070989973
Average COMET-MQM for 't5': 0.03162484507386883
Average COMET for 't5': 0.8057184248794744


In [45]:
for key, val in da.items():
    print(f"{key} : {val:.4f}")

original : -0.0009
only_punct : -0.0176
without_punct : -0.0258
combined : -0.0118
combined_le : -0.0080
combined_led : -0.0108
gpt : -0.0494
bert : 0.0114
mpnet : 0.0138
t5 : 0.0158


In [46]:
for key, val in mqm.items():
    print(f"{key} : {val:.4f}")

original : 0.0304
only_punct : 0.0295
without_punct : 0.0289
combined : 0.0295
combined_le : 0.0296
combined_led : 0.0295
gpt : 0.0280
bert : 0.0310
mpnet : 0.0313
t5 : 0.0316


In [47]:
for key, val in comet.items():
    print(f"{key} : {val:.4f}")

original : 0.7815
only_punct : 0.7733
without_punct : 0.7709
combined : 0.7749
combined_le : 0.7837
combined_led : 0.7844
gpt : 0.7372
bert : 0.8049
mpnet : 0.8060
t5 : 0.8057
